In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from os import listdir, makedirs
from os.path import isfile, join, exists
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from torch.autograd import Variable
%matplotlib
%pylab inline

plt.ion()   # interactive mode

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\hl2278\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
from torch.utils import data
from PIL import Image

def simple_loader(img_path):
    image = Image.open(img_path).convert("RGB")
    return image

class AerialImageDataset(data.Dataset):
    def __init__(self, transform=None, set_option="train", loader=simple_loader):
        img_list, target_list = self.make_dataset(set_option)
#         self.root = data_root
        self.img_list = img_list
        self.target_list = target_list
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        img_path, target = self.img_list[index], self.target_list[index]
        img = self.loader(img_path)
        img = self.transform(img)
        return img, torch.LongTensor([int(target)])

    def __len__(self):
        return len(self.img_list)
    
    def make_dataset(self, set_option):
        wd = os.path.join(os.getcwd(),"train")
        files = [f for f in listdir(wd) if isfile(join(wd, f))]
        img_list = []
        for tf in files:
            tfwd = os.path.join("train", tf)
            img_list.append(os.path.join(os.getcwd(), tfwd))
            
        target_list = pickle.load(open("trainlabels.pickle", "rb"))
        
        if set_option == "train":
            target_list = target_list[:63000]
            img_list = img_list[:63000]
        else:
            target_list = target_list[80000:]
            img_list = img_list[80000:]
            
        return img_list, target_list

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = AerialImageDataset(data_transforms["train"], "train")
test_dataset = AerialImageDataset(data_transforms["val"], "test")

In [4]:
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_dataloaders = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0)

In [ ]:
img, target = train_dataset[0]

In [5]:
class ResNet50(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        resnet = models.resnet50(pretrained)

        self.layer0 = nn.Sequential(
            resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool
        )
        self.layer1, self.layer2, self.layer3, self.layer4, self.avgpool = resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4, nn.AvgPool2d(2)
        self.fc = nn.Linear(18432, 14951)
        
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        
        return self.fc(x)

In [6]:
lr=0.01

model = ResNet50().cuda()
model.train()

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay=0.0001)

max_epochs, number_of_epochs, best_accuracy, previous_epoch = 100, 0, 0.0, -1

criterion = nn.CrossEntropyLoss().cuda()

In [7]:
class AverageMeter(object):
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def validate(data_loader, model, criterion, epoch):
    model.eval()
    
    top1_m = AverageMeter()    
    for vi, data in enumerate(data_loader):
        
        images, labels = data
        images = Variable(images, requires_grad=False, volatile=True).cuda()
        labels_var = Variable(labels, requires_grad=False, volatile=True).cuda()
        
        predictions = model(images)

        # Val Loss - to check the learning
        # loss = criterion(predictions, labels_var)
        prec1, prec5 = accuracy(predictions.data.cpu(), labels, topk=(1, 5))
        top1_m.update(prec1[0], images.size(0))    
    model.train()
    return top1_m.avg

def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def save_checkpoint(state, is_best, model_specs, model_description):    
    filename = os.path.join(os.getcwd(), "_".join(['chkpt', model_description]) + '.pth.tar')
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(os.getcwd(), "_".join(['best_model', model_description]) + '.pth.tar'))
        json.dump(model_specs, open(os.path.join(os.getcwd(), "_".join(['best_model_metrics', model_description]) + '.json'), "w"))

train_accuracy = []
val_accuracy = [] 

for epoch in range(0, 200):
    top1_m = AverageMeter()    
    for vi, data in tqdm(enumerate(train_dataloaders)):
#         optimizer.param_groups[0]['lr'] = lr * (1-float(epoch)/max_epochs) ** lr_decay
        images, labels = data
        images = Variable(images, requires_grad=False).cuda()
        labels_var = Variable(labels, requires_grad=False).cuda()

        predictions = model(images)
        
        loss = criterion(predictions, labels_var.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        prec1 = accuracy(predictions.data.cpu(), labels, topk=(1, ))
        top1_m.update(prec1[0], images.size(0))
        if vi % 100 == 0:
            print(loss)
        
    train_accuracy.append(top1_m.avg.numpy()[0])    
    val_accuracy.append(validate(test_dataloaders, model, criterion, epoch))
    print(val_accuracy[-1])
    save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': val_accuracy[-1],
                'optimizer' : optimizer.state_dict(),
            }, True, {"epoch" : epoch + 1, "accuracy" : val_accuracy[-1]}, "model")

0it [00:00, ?it/s]

Variable containing:
 9.8898
[torch.cuda.FloatTensor of size 1 (GPU 0)]



100it [00:22,  4.51it/s]

Variable containing:
 8.2401
[torch.cuda.FloatTensor of size 1 (GPU 0)]



200it [00:43,  4.64it/s]

...

7800it [27:30,  4.73it/s]

Variable containing:
 6.6980
[torch.cuda.FloatTensor of size 1 (GPU 0)]



7875it [27:46,  4.73it/s]


18.97003531958528


RuntimeError: unknown error